### Explore the 'overflow' warning/error from the 'CARMATerm.py' in the fitting

<br>**Author(s):** Weixiang Yu
<br>**Last run:** 08-21-20
<br>**Short description:** Explore the 'overflow' warning/error from the 'CARMATerm.py' in the fitting

In [1]:
# import basic packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys

# see if local stores mpl style, else use from src
try:
    plt.style.use('yu_basic')
except:
    mpl.rc_file('https://raw.githubusercontent.com/ywx649999311/project_template'
                '/master/%7B%7Bcookiecutter.project_name%7D%7D/src/vis/mpl/yu_basic.rc')

pd.set_option('display.max_columns', 999)
%matplotlib inline

In [2]:
# import CARMA celerite term
from agntk.carma.CARMATerm import *
from agntk.lc.utils import *
from celerite import GP
from scipy.optimize import minimize
import celerite

### 1. Simulate DHO LC

In [3]:
# define GP model
a1 = 0.03939692
a2 = 0.00027941
b0 = 0.0046724
b1 = 0.0256982
kernel = DHO_term(np.log(a1), np.log(a2), np.log(b0), np.log(b1))
gp = GP(kernel)

# check if Celerite give the same amp as Kali
assert np.allclose(1.0, kernel.get_amp())

In [4]:
# now simulate 10 years LC
npts = 100
nLC = 50

# init with dense -> add season -> downsample
t_init = np.linspace(0, 3650, 10000)
yerr_init = np.random.normal(0, 1e-7, 10000) # give very tiny error
gp.compute(t_init, yerr_init)
y_init = gp.sample(size=nLC)

tArr = np.zeros((nLC, npts))
yArr = np.zeros((nLC, npts))
yerrArr = np.zeros((nLC, npts))

for i in range(nLC):    
    mask1 = add_season(t_init)
    mask2 = downsample_byN(t_init[mask1], npts)
    tArr[i,:] = t_init[mask1][mask2]
    yArr[i,:] = y_init[i][mask1][mask2]
    yerrArr[i,:] = yerr_init[mask1][mask2]


print(f'Mean Sim Std: {np.mean(np.std(yArr, axis=1))}')
print(f'True Std: {1}')

Mean Sim Std: 0.9455072303426683
True Std: 1


### 2. Search for the error

In [25]:
def neg_log_like_1(params, y, yerr, gp):
    np.seterr(over='raise')
    np.seterr(under='raise')
    params = np.array(params)
    run = True
    lap = 0
    try:
        while run:
            if lap > 50:
                return -np.inf

            lap += 1
            try:
                gp.set_parameter_vector(params)
                neg_ll = -gp.log_likelihood(y)
                run=False
            except celerite.solver.LinAlgError:
                params += 1e-6*np.random.randn(4)
                continue
            except np.linalg.LinAlgError:
                params += 1e-6*np.random.randn(4)
                continue
    except FloatingPointError as e:
        print(params)
        print(e)
#         return -np.inf
    return neg_ll

def neg_log_like_2(params, y, yerr, gp):
    np.seterr(over='raise')
    np.seterr(under='raise')
    params = np.array(params)
    run = True
    lap = 0
    try:
        while run:
            if lap > 50:
                return -np.inf

            lap += 1
            try:
                gp.set_parameter_vector(params)
                neg_ll = -gp.log_likelihood(y)
                run=False
            except celerite.solver.LinAlgError:
                params += 1e-6*np.random.randn(4)
                continue
            except np.linalg.LinAlgError:
                params += 1e-6*np.random.randn(4)
                continue
    except FloatingPointError as e:
        return -np.inf
    return neg_ll

In [26]:
## start fitting
for i in range(50):

    # initialize parameter in the possible range
    a1 = np.exp(np.random.uniform(-3, 1, 1)[0])
    a2 = np.exp(np.random.uniform(-6, 1, 1)[0])
    b0 = np.exp(np.random.uniform(-10, -5, 1)[0])
    b1 = np.exp(np.random.uniform(-10, -5, 1)[0])

    kernel_fit = DHO_term(np.log(a1), np.log(a2), np.log(b0), np.log(b1))
    gp_fit = GP(kernel_fit)
    gp_fit.compute(tArr[i], yerrArr[i])

    initial_params = gp_fit.get_parameter_vector()
    bounds = gp.get_parameter_bounds()

    ret = minimize(neg_log_like_1, initial_params, method="L-BFGS-B", bounds=bounds, \
                 args=(yArr[i], yerrArr[i], gp_fit))

[ -49.33769902 -107.3207002   -46.53760524  819.52104745]
overflow encountered in exp


UnboundLocalError: local variable 'neg_ll' referenced before assignment

<span style='color:red'>**Remark:**</span> It is clear that the `RuntimeWarning` is caused by overly large/small numbers suggested by the optimizing alogorithms. There are two solutions, one is adding a clause the in the likelihood function to catch overflow/underflow and return -inf; another way to handle this is to specify a boundary in the optimization algorithms for making proposals, which would be highly algorithm dependent.  

#### Next, Let's run the above code again with the frist solution

In [29]:
## start fitting
for j in range(100):
    for i in range(50):

        # initialize parameter in the possible range
        a1 = np.exp(np.random.uniform(-3, 1, 1)[0])
        a2 = np.exp(np.random.uniform(-6, 1, 1)[0])
        b0 = np.exp(np.random.uniform(-10, -5, 1)[0])
        b1 = np.exp(np.random.uniform(-10, -5, 1)[0])

        kernel_fit = DHO_term(np.log(a1), np.log(a2), np.log(b0), np.log(b1))
        gp_fit = GP(kernel_fit)
        gp_fit.compute(tArr[i], yerrArr[i])

        initial_params = gp_fit.get_parameter_vector()
        bounds = gp.get_parameter_bounds()

        ret = minimize(neg_log_like_2, initial_params, method="L-BFGS-B", bounds=bounds, \
                     args=(yArr[i], yerrArr[i], gp_fit))

<span style='color:red'>**Conclusion:**</span> After running the above loop for 100 times, no additional warning is raised, it seems like the suggested fix worked. 
<span style='color:red'>**Note:**</span> It appears that the `basin-hopping` opitimizer worker better (giving solutions with smaller errors with the fix, but will require longer runs